In [7]:
import os

DATA_DIR = "data" # This may need to be changed on different machines

# Make sure we're in the correct directory and make sure the data directory exists
if not os.path.exists(DATA_DIR):
    os.chdir("../..") # Move up two directories because we're in src/nb and the data directory/path should be in/start at the root directory 
    assert os.path.exists(DATA_DIR), f"ERROR: DATA_DIR={DATA_DIR} not found"  # If we still can't see the data directory something is wrong


from torch.utils.data import DataLoader

import pandas as pd
from src.lib.decoder_dataset import DecoderDataset


In [8]:
save_path = os.path.join(DATA_DIR, "decoded_cds", "balanced", "dev_dataset.pth")

In [9]:
if not os.path.exists(save_path):
    df = pd.read_csv(os.path.join(DATA_DIR, "decoded_cds", "balanced", "dev.csv"), index_col=0)
    dataset = DecoderDataset(df)

    dataset.save_state_dict(save_path)
else:
    dataset = DecoderDataset.from_state_dict(save_path)

In [10]:
print(len(dataset))

14443


In [11]:

data_loader = DataLoader(dataset, batch_size=20, shuffle=True)

for batch in data_loader:
    x, y = batch
    style_encoding, para, bos_pos, target, attn_mask = x
    para_embed, para_pos = para
    target_embeds, target_pos = target
    label, label_idx = y
    
    print(style_encoding.shape)

    print(para_embed.shape)
    print(para_pos.shape)

    print(bos_pos.shape)

    print(target_embeds.shape)
    print(target_pos.shape)

    print(attn_mask.shape)
    
    print(label.shape)
    print(label_idx.shape)

    break
    

torch.Size([20, 768])
torch.Size([20, 12, 1280])
torch.Size([20, 12, 1280])
torch.Size([20, 1280])
torch.Size([20, 12, 1280])
torch.Size([20, 12, 1280])
torch.Size([20, 26])
torch.Size([20])
torch.Size([20])


In [12]:
batch = dataset[0]
x, y = batch
style_encoding, para, bos_pos, target, attn_mask = x
para_embed, para_pos = para
target_embeds, target_pos = target
label, label_idx = y

print(attn_mask)
print(para_embed.sum(dim=1))
print(label_idx, label)


tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])
tensor([ 1.2377,  1.5913, -0.2598,  1.2065,  0.3681, -0.1464, -0.4857,  0.1138,
         0.3017,  0.4296, -0.3893,  0.7515], grad_fn=<SumBackward1>)
16 tensor(11)
